In [16]:
import numpy as np
from tensorflow.keras.models import load_model
import cv2
import mediapipe as mp

In [17]:
model = load_model('b.h5', compile=False)

In [18]:



def image_processed(file_path):
    # Reading the static image
    # hand_img = cv2.imread(file_path)

    # Image processing
    # 1. Convert BGR to RGB
    img_rgb = cv2.cvtColor(file_path, cv2.COLOR_BGR2RGB)

    # 2. Flip the img in Y-axis
    img_flip = cv2.flip(img_rgb, 1)

    # Accessing MediaPipe solutions
    mp_hands = mp.solutions.hands

    # Initialize Hands
    hands = mp_hands.Hands(static_image_mode=True,
                           max_num_hands=1,
                           min_detection_confidence=0.8,
                           min_tracking_confidence=0.7)

    # Results
    output = hands.process(img_flip)

    hands.close()

    try:
        data = output.multi_hand_landmarks[0]
        print(data)

        data = str(data)

        data = data.strip().split('\n')

        garbage = ['landmark {', '  visibility: 0.0', '  presence: 0.0', '}']

        without_garbage = []

        for i in data:
            if i not in garbage:
                without_garbage.append(i)

        clean = []

        for i in without_garbage:
            i = i.strip()
            clean.append(i[2:])

        for i in range(0, len(clean)):
            clean[i] = float(clean[i])

        return clean

    except:
        return np.zeros([1, 63], dtype=int)[0]




In [22]:
import cv2
import numpy as np

# Open camera
y_target = ['paper', 'scissor', 'rock', 'nothing']
cap = cv2.VideoCapture(0)

width, height = 1280, 720
x, y, w, h = 100, 100, 400, 400

if not cap.isOpened():
    print("Cannot open camera")
    exit()

# Load background detection model (e.g., MOG2)
background_model = cv2.createBackgroundSubtractorMOG2()

# Threshold value for classifying "nothing"
threshold_value = 5000

while True:

    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (width, height))

    # Apply cv2.rectangle within the frame
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    frame_roi = frame[y:y + h, x:x + w]  # Region of Interest

    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Apply background subtraction to detect foreground objects
    foreground_mask = background_model.apply(frame_roi)

    # Perform image processing on the foreground mask (e.g., thresholding)
    _, threshold_mask = cv2.threshold(foreground_mask, 128, 255, cv2.THRESH_BINARY)

    # Count the number of white pixels in the threshold mask
    white_pixel_count = np.sum(threshold_mask == 255)

    if white_pixel_count < threshold_value:
        # If the number of white pixels is below the threshold, classify as "nothing"
        target_index = len(y_target) - 1
        prob = 1.0
    else:
        # Perform prediction on the ROI as before
        data = image_processed(frame_roi)
        data = np.array(data, dtype=np.float32)
        y_pred = model.predict(data.reshape(-1, 63))
        target_index = np.argmax(y_pred[0])
        prob = np.max(y_pred[0])

    # Display prediction on the frame
    text = f'Predict {y_target[target_index]} with {np.round(prob * 100)}%'
    cv2.putText(frame, text, (0, 400), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_8, False)
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 19ms/step
landmark {
  x: 0.4975175
  y: 0.65736884
  z: 5.388984e-07
}
landmark {
  x: 0.28128508
  y: 0.6463973
  z: -0.04480883
}
landmark {
  x: 0.10685976
  y: 0.5392711
  z: -0.05358935
}
landmark {
  x: 0.025896534
  y: 0.422426
  z: -0.059160974
}
landmark {
  x: -0.031548977
  y: 0.33378947
  z: -0.059811704
}
landmark {
  x: 0.16870019
  y: 0.30433115
  z: -0.0043256073
}
landmark {
  x: 0.08642802
  y: 0.16424929
  z: -0.02991386
}
landmark {
  x: 0.054608062
  y: 0.077742174
  z: -0.059751075
}
landmark {
  x: 0.028864235
  y: 0.016799778
  z: -0.08330355
}
landmark {
  x: 0.29037592
  y: 0.24817075
  z: -0.012741143
}
landmark {
  x: 0.22347042
  y: 0.09153402
  z: -0.029057281
}
landmark {
  x: 0.17774561
  y: 0.0072404444
  z: -0.05704003
}
landmark {
  x: 0.13391066
  y: -0.04359001
  z: -0.077802815
}
landmark {
  x: 0.41680253
  y: 0.23920467
  z: -0.031044427
}
landmark {
  x: 0.37526292
  y: 0.08334164
  z: -0.05808621
}
lan